# Imports

In [1]:
import re
from bs4 import BeautifulSoup
import os

# Global Values

# Splitting algorithms

In [12]:
# Blockquotes are not saved
def split_2009_until_today(url):
    with open(url) as file:
        soup = BeautifulSoup(file.read(), 'html.parser')
    date = "".join(re.findall('\d\d\d\d\d+', url))
    if int(date[:4]) >= 2012:
        contents = soup.find(id = "article").find_all(re.compile("p|blockquote"))
    else:
        contents = soup.find(id = "leftText").find_all(re.compile("p|blockquote"))
    contents_cleaned = []
    section = ''
    section_set = False
    help_20091104 = False
    for i in range(0, len(contents)):  
        if date != "20091104":
            if re.search('<strong>', str(contents[i])):
                if re.search('Open Market|Financial Market', str(contents[i].find("strong")), re.IGNORECASE):
                    section = 'Open Market'
                    section_set = True
                elif re.search('Staff Review|Inflation Analysis|Structural Unemployment', str(contents[i].find("strong")), re.IGNORECASE):
                    section = 'Staff Review'
                    section_set = True
                elif re.search('Participants', str(contents[i].find("strong")), re.IGNORECASE):
                    section = 'FOMC members'
                    section_set = True
                elif re.search('Committee Policy Action', str(contents[i].find("strong")), re.IGNORECASE):
                    section = 'Policy'
                    section_set = True
                elif re.search('Voting', str(contents[i].find("strong")), re.IGNORECASE) or re.search('Vote', str(contents[i].find("strong")), re.IGNORECASE):
                    break
        else:
            if re.search('<b>', str(contents[i])):
                if re.search('Developments in Financial Markets and the Federal Reserve', str(contents[i].find("b")), re.IGNORECASE):
                    if help_20091104 == False:
                        help_20091104 = True
                        continue
                    section = 'Open Market'
                    section_set = True
                elif re.search('Staff Review|Inflation Analysis|Structural Unemployment', str(contents[i].find("b")), re.IGNORECASE):
                    section = 'Staff Review'
                    section_set = True
                elif re.search('Participants', str(contents[i].find("b")), re.IGNORECASE):
                    section = 'FOMC members'
                    section_set = True
                elif re.search('Committee Policy Action', str(contents[i].find("b")), re.IGNORECASE):
                    section = 'Policy'
                    section_set = True
                elif re.search('Voting', str(contents[i].find("b")), re.IGNORECASE) or re.search('Vote', str(contents[i].find("b")), re.IGNORECASE):
                    break
        if section != '':
            string = contents[i].get_text().strip()
            string = re.sub('\n', '.', string)
            string = re.sub(' \.', '', string)
            string = re.sub(' \.', '', string)
            string = re.sub(' +', ' ', string)
            if section_set == True:
                string = string.split(".")[1:]
                string = ''.join(string)
                section_set = False
            if int(date[:4]) <= 2011:
                if re.search("discussion", string) and re.search("members", string) and section == "FOMC members":
                    section = 'Policy'
            if len(string) > 200 and len(string) < 10000:
                if re.search("In conducting operations pursuant to the authorization", string):
                    section = ''
                    continue
                contents_cleaned.append((section, string)) 
    return contents_cleaned

In [ ]:
def split_2008_and_older(url):
    with open(url) as file:
        soup = BeautifulSoup(file.read(), 'html.parser')
    date = "".join(re.findall('\d\d\d\d\d+', url))
    contents = soup.find_all("p")
    contents_split = []
    contents_cleaned = []
    section = ''
    number_sections_set = 0
    counter = 1
    for p in contents:
        counter = counter + 1
        text = p.get_text().strip()
        if number_sections_set == 0:
            if text.startswith("The Manager of the System Open Market Account"):
                section = 'Open Market'
                number_sections_set = 1
                
        elif number_sections_set == 1:
            if 'forecast' in text and 'staff' in text:
                section = 'Staff Review'
                number_sections_set = 2
                
        elif number_sections_set == 2:
            if 'participants' in text and 'economic situation' in text or 'economic outlook' in text:
                section = 'FOMC members'
                number_sections_set = 3
                
        elif number_sections_set == 3:
            if 'Committee' in text and 'monetary policy' in text:
                section = 'Policy'
                number_sections_set = 4
                
        elif number_sections_set == 4:
            if 'Notation Vote' in text:
                section = ''
        if section != '':
            if len(text) > 200 and len(text) < 10000:
                text = re.sub(r'\x92', '', text)
                text = re.sub(r'\x93', '', text)
                text = re.sub(r'\x94', '', text)
                text = re.sub('\n', '.', text)
                text = re.sub(' \.', '', text)
                text = re.sub(' \.', '', text)
                text = re.sub(' +', ' ', text)
                contents_cleaned.append((section, text)) 
    if number_sections_set < 4:
        print("Nicht alle Sektionen gesetzt, Fehler bei Sektion " + str((number_sections_set + 1)))
    return contents_cleaned

In [15]:
files = os.listdir("minutes_html")
for f in files:
    for r in range(2009, 2019):
        if f.startswith(str(r)):
            x = split_2009_until_today("minutes_html/" + f)
            string = f + ": ["
            counter1 = 0
            counter2 = 0
            counter3 = 0
            counter4 = 0
            for i in x:
                if i[0] == "Open Market":
                    counter1 = counter1 + 1
                elif i[0] == "Staff Review":
                    counter2 = counter2 + 1
                elif i[0] == "FOMC members":
                    counter3 = counter3 + 1
                elif i[0] == "Policy":
                    counter4 = counter4 + 1
            string = string + str(counter1) + "," + str(counter2) + "," + str(counter3) + "," + str(counter4) + "]"
            print(string)

20090128_minutes_html.txt: [7,22,0,16]
20090318_minutes_html.txt: [8,20,8,7]
20090429_minutes_html.txt: [5,21,11,7]
20090624_minutes_html.txt: [14,22,9,8]
20090812_minutes_html.txt: [4,20,9,8]
20090923_minutes_html.txt: [6,20,8,6]
20091104_minutes_html.txt: [4,19,10,8]
20091216_minutes_html.txt: [3,20,10,8]
20100127_minutes_html.txt: [8,18,7,7]
20100316_minutes_html.txt: [6,21,7,9]
20100428_minutes_html.txt: [10,21,10,7]
20100623_minutes_html.txt: [4,25,8,7]
20100810_minutes_html.txt: [2,23,9,9]
20100921_minutes_html.txt: [1,23,7,8]
20101103_minutes_html.txt: [2,24,8,10]
20101214_minutes_html.txt: [3,27,8,9]
20110126_minutes_html.txt: [1,26,10,9]
20110315_minutes_html.txt: [2,28,9,11]
20110427_minutes_html.txt: [8,27,12,11]
20110622_minutes_html.txt: [5,29,9,8]
20110809_minutes_html.txt: [1,24,9,10]
20110921_minutes_html.txt: [6,26,9,12]
20111102_minutes_html.txt: [5,26,9,9]
20111213_minutes_html.txt: [1,26,7,11]
20120125_minutes_html.txt: [1,26,11,12]
20120313_minutes_html.txt: [1,25,